In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/My Drive/recsys-hw2/train.csv')
songs = pd.read_csv('/content/drive/My Drive/recsys-hw2/songs.csv')

In [4]:
train.head(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
songs.head(5)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [6]:
train.drop(['source_system_tab', 'source_screen_name', 'source_type'], axis=1, inplace=True)

train = train[train['target'] == 1]

target = train['target']

train.drop(['target'], axis=1, inplace=True)

In [7]:
temp = train.groupby(['msno'])['song_id'].apply(list)
temp = temp.to_frame().reset_index()

temp.columns = ['msno', 'song_id_history']

histories = list(temp['song_id_history'])

print(len(histories[0]))

293


Для каждого пользователя вытащили историю его лайков, обучим на них skipgram. Взял не очень большие размер эмбеддинга (64) и окно (5), чтобы училось чуть-чуть быстрее.

In [8]:
%%time

from gensim.models import Word2Vec

model = Word2Vec(window=5, sg=1, hs=0, size=64, negative=15, alpha=0.03, min_alpha=0.0007)
model.build_vocab(histories, progress_per=100)

model.train(histories, total_examples=model.corpus_count, epochs=10, report_delay=1)

CPU times: user 15min 24s, sys: 750 ms, total: 15min 24s
Wall time: 5min 19s


In [9]:
print(model[model.wv.vocab].shape)

(55816, 64)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [10]:
print(model[model.wv.vocab][0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


[-0.1386644   0.16811465 -0.07710329  0.10071291 -0.10733915  0.15520175
  0.09295902  0.24388668 -0.32286507  0.21121305  0.24251153 -0.10480519
 -0.01494531  0.73905617 -0.37164432 -0.36116302 -0.4505414   0.05940901
 -0.05257434  0.25837684  0.95397     0.00588339  0.12667018  0.13955326
  0.2579292  -0.4317106  -0.75939804  0.51084507  0.36821386 -0.2868113
 -0.35827813 -0.09445771  0.44319093  0.28712735  0.3391168   0.30366856
 -0.22723241  0.33926582 -0.14840923 -0.14498012 -0.40942743 -0.24402346
  0.00535069  0.00885501  0.51714456 -0.34546134 -0.35436046 -0.38189074
 -0.47600317 -0.24611379  0.4151743  -0.6885147   0.3859774  -0.05989029
  0.31133682 -0.22801706  0.91279787  0.44986704  0.03310683  0.22453432
 -0.6975126  -0.77596074 -1.0273355   0.17932817]


In [11]:
model.init_sims(replace=True)

In [12]:
def get_similars(initial_song_id, k=20):
    similars = model.similar_by_vector(model[initial_song_id], topn=k + 1)
    res = []
    
    for song_id, similarity in similars:
        row = songs[songs['song_id'] == song_id].to_dict('records')[0]
        row['similarity'] = similarity
        res.append(row)
        
    return pd.DataFrame(res)

In [13]:
initial_song_id = list(model.wv.vocab.items())[4000][0]
get_similars(initial_song_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,similarity
0,7HDcHFAVdw76mL4Z5hbz300+s1oSUstaOfmaLw0+Dfo=,225256,465,Rihanna,Mikkel S. Eriksen| Sia Furler| B Levin| T.E. H...,NaN,52.0,1.000000
1,u9z+kvlYCkrSlDdJv3O/t5h+JpZJrSn2ZBI1GWe+EYQ=,202013,465,Miley Cyrus,Lukasz Gottwald| Claude Kelly| Jessica Cornish,NaN,52.0,0.861619
2,2AEXIjiK5meckRxl3k3zv5FnpimIvBrtgsI6FyFOlFI=,243461,465,Rihanna,Mikkel S. Eriksen| Tor Erik Hermansen| Sandy W...,NaN,52.0,0.824449
3,J8poi1Mg9pKUq7Kw/w9y7JVmX3AH5OeFh4YJJi3hieY=,200341,465,Nicki Minaj,Nicki Minaj| D. Johnson| E. Dean,NaN,52.0,0.820467
4,4/f7S7iNOygpoPNxUzMv7jOPLg4HGYhpbEO7i41zUzU=,265142,465,Nelly Furtado,NaN,NaN,52.0,0.813892
5,VFWmrWVR8IbIiP4iwS+RwATCVRqZozRMJ06PqQQeEBI=,215202,465,Rihanna,Calvin Harris,NaN,52.0,0.806797
6,uuv/oI/dfOrHOHwDN9co6xyS484UCmel5TPfqOWCiU8=,296124,465,Rihanna,Alexander Grant| Skylar Grey| Marshall Mathers,NaN,52.0,0.803231
7,ES86JRfxcI9nxEfNv70Y772KQc9gxpykvAD7XRTsBgM=,284467,465,Will.I.Am,Jef Martens| William Adams| Jean Baptiste,NaN,52.0,0.792608
8,0zmrdr16SIqvo1e4ss8cp+BbkkYKCFQ/RT+iouvGJmc=,232578,465,Miley Cyrus,Miley Cyrus| Mike L. Williams II| Pierre Ramon...,Miley Cyrus| Mike L. Williams II| Pierre Ramon...,52.0,0.785390
9,GcnaI018MEvQeZHYZMYlzFbMDc+2GU76kxh53yanra4=,275945,465|139,Rihanna,Thaddis Harrell| Shawn Carter,NaN,52.0,0.782912


Посмотрим на симилары. К Рианне советуется разнообразная попса (включая саму Рианну). Уже неплохо.

In [14]:
song_embeddings = pd.DataFrame(model[model.wv.vocab])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [15]:
song_embeddings['song_id'] = np.array(list(model.wv.vocab.keys()))

In [16]:
# song_embeddings.to_csv('/content/drive/My Drive/recsys-hw2/song_embeddings.csv')

In [17]:
song_embeddings.set_index('song_id', inplace=True)

song_embeddings.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
song_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
aRNy4rKgWyDwTjsMIcAbtpGvddDZ7L7OGUeJ09LAM2w=,-0.043964,0.053301,-0.024446,0.031931,-0.034032,0.049207,0.029473,0.077325,-0.102366,0.066966,0.076889,-0.033229,-0.004738,0.234320,-0.117831,-0.114508,-0.142846,0.018836,-0.016669,0.081919,0.302460,0.001865,0.040161,0.044246,0.081777,-0.136875,-0.240770,0.161965,0.116743,-0.090935,-0.113593,-0.029948,0.140515,0.091035,0.107518,0.096279,-0.072045,0.107565,-0.047054,-0.045966,-0.129810,-0.077368,0.001696,0.002808,0.163962,-0.109530,-0.112351,-0.121080,-0.150918,-0.078031,0.131632,-0.218296,0.122375,-0.018988,0.098710,-0.072294,0.289406,0.142632,0.010497,0.071189,-0.221149,-0.246021,-0.325720,0.056857
J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=,-0.230938,0.058755,0.050191,-0.022890,-0.023038,0.051715,-0.025400,0.165169,-0.051146,-0.003103,0.208125,-0.089423,0.228371,0.060925,-0.105263,-0.076140,-0.040101,-0.015691,0.147345,0.108621,0.124137,-0.144734,0.013060,-0.108929,0.044904,-0.113979,-0.175315,0.178227,0.088189,-0.099434,-0.154270,-0.038105,0.193143,0.134842,0.033705,0.159449,-0.027802,0.050728,-0.038280,0.128446,-0.118008,0.023012,-0.081440,0.121514,-0.014028,-0.023213,-0.162675,-0.165648,-0.021198,-0.036172,0.189413,-0.283284,0.261416,-0.113110,0.007918,-0.061760,0.180420,0.006322,0.083636,0.112686,-0.083473,-0.302735,-0.151975,0.086257
zHqZ07gn+YvF36FWzv9+y8KiCMhYhdAUS+vSIKY3UZY=,-0.279672,0.042649,-0.071112,0.063093,-0.005431,0.069160,0.043952,0.139852,-0.095331,0.012578,0.178014,-0.044374,0.212763,0.121827,-0.078254,-0.070388,-0.060200,0.046001,0.097421,0.076268,0.173474,-0.188830,0.031008,-0.067771,0.111382,-0.122954,-0.169940,0.159448,0.051148,-0.033920,-0.173908,-0.019696,0.231088,0.095005,0.060377,0.155311,-0.070750,0.084411,-0.074738,0.101438,-0.065813,-0.002334,-0.099328,0.099674,-0.004498,-0.018825,-0.205598,-0.205359,-0.059856,-0.009027,0.145669,-0.243018,0.224808,-0.161663,0.072910,-0.105862,0.188432,-0.007918,0.044983,0.087105,-0.073438,-0.281245,-0.167240,0.004345
v/3onppBGoSpGsWb8iaCIO8eX5+iacbH5a4ZUhT7N54=,-0.323799,0.033191,-0.067498,0.006102,-0.090865,0.036851,0.064599,0.060751,-0.135431,0.083040,0.158339,-0.235818,0.243387,0.071732,-0.099041,-0.053988,-0.016199,0.099335,0.054181,0.175597,0.173612,-0.137518,0.044578,0.046649,0.082111,-0.169591,0.048294,0.208751,0.061873,-0.129331,-0.094219,-0.011473,0.156812,0.069027,-0.054116,0.157835,-0.009178,0.093943,-0.119666,0.130018,-0.075325,-0.087021,-0.069202,0.069796,0.060150,-0.075806,-0.137552,-0.168882,-0.017549,0.006209,0.091693,-0.200792,0.255150,-0.141018,0.078656,-0.095745,0.119316,0.095194,0.126524,0.156238,-0.147150,-0.167881,-0.200243,-0.027545
upH6pOAUd+iV/MkpzeELvqEFoTEIVhsV9eML8N7/gUM=,-0.378116,-0.044505,-0.055448,0.001135,-0.044965,-0.061248,-0.081650,0.142906,0.087215,-0.016871,0.091886,-0.100535,0.334319,0.157084,-0.069582,0.047250,0.009750,-0.160114,0.048810,0.267818,0.164001,-0.073963,-0.021030,-0.116707,-0.123987,-0.074273,-0.082374,0.086565,-0.032545,-0.078607,-0.330850,-0.012228,0.119139,0.141622,0.079158,-0.143018,0.146931,0.084485,-0.011997,-0.186052,0.057524,-0.086052,0.124342,-0.033767,-0.037614,0.099659,-0.175714,0.013859,-0.061866,0.005887,0.115327,-0.049318,0.154488,-0.091979,0.049436,0.086175,0.150619,0.076529,-0.123591,-0.023233,-0.073708,0.039863,-0.202070,-0.051693


Посчитаем эмбеддинг юзера как среднее арифметическое эмбеддингов послушанных им песен.

In [18]:
%%time

from collections import defaultdict

def get_user_embeddings(df):
    user_embeddings = defaultdict(lambda: np.zeros(64))
    user_counts = defaultdict(int)

    for (_, row) in df.iterrows():
        user_id = row['msno']
        song_id = row['song_id']

        if song_id in model.wv:
            user_embeddings[user_id] += model.wv[song_id]
            user_counts[user_id] += 1

    for user_id in user_embeddings.keys():
        user_embeddings[user_id] /= user_counts[user_id]

    return user_embeddings

res = get_user_embeddings(train)

CPU times: user 5min 51s, sys: 3.85 s, total: 5min 55s
Wall time: 5min 49s


In [19]:
res = pd.DataFrame(res)
res = res.T

res.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,-0.099113,0.018728,-0.044590,-0.021446,-0.032362,-0.001739,-0.008489,0.022638,0.000116,-0.076708,0.110048,-0.124888,0.057543,0.166530,0.077098,-0.054071,-0.075878,-0.111791,0.077787,0.141790,0.116717,-0.107845,-0.052120,-0.049851,-0.031012,-0.124564,-0.020556,0.143448,0.002290,0.062035,-0.170154,0.049212,0.054290,0.114577,-0.001245,-0.037983,0.035608,0.094007,-0.092774,0.022830,-0.100738,-0.032309,0.006536,0.009156,0.035708,-0.043198,-0.145385,-0.089903,-0.046610,-0.091211,0.121065,-0.124360,0.101118,-0.021641,-0.083355,-0.075402,0.055138,0.059780,0.018694,-0.003879,-0.028879,-0.093264,-0.069001,-0.002445
Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,-0.114728,0.035292,0.024608,-0.041625,-0.012095,-0.034106,-0.067257,0.049292,-0.037195,-0.061133,0.022139,-0.148769,0.068678,0.109887,0.081286,-0.036582,-0.046270,-0.088054,0.032982,0.187044,-0.003347,0.004465,-0.043814,-0.131041,0.000877,-0.058584,-0.077218,0.157328,0.059829,0.007036,-0.135923,0.038799,0.029293,0.093565,0.035968,0.048454,0.014236,0.086094,-0.162544,0.049051,-0.113638,0.023439,0.025061,0.114936,0.013844,-0.035336,-0.195187,-0.096994,-0.098608,-0.094757,0.184940,-0.075562,0.146126,-0.101707,-0.023295,-0.071530,0.067280,0.095334,-0.016725,-0.012278,-0.067520,0.007159,-0.050208,-0.029487
uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=,-0.120721,0.053595,-0.045988,-0.046414,0.044853,0.027565,-0.031272,0.049844,-0.045903,-0.030648,0.084649,-0.141983,0.107872,0.123807,0.057164,-0.001455,-0.004299,-0.092720,0.049538,0.209054,0.100056,-0.125803,-0.059491,-0.065860,0.061796,-0.131193,-0.038506,0.098504,0.028440,0.006569,-0.196350,0.034595,0.027298,0.119728,0.012213,-0.020240,-0.001929,0.058732,-0.113211,-0.029914,-0.031651,0.046856,-0.019791,0.007424,0.013564,-0.055065,-0.174027,-0.106123,-0.057505,-0.067711,0.115889,-0.067298,0.125817,-0.027097,-0.020332,-0.051018,0.155010,0.055361,-0.018133,-0.042904,-0.042838,-0.062552,-0.073814,-0.005229
3g0bC24RD7QUeALY1IF/e8L6X3fHG298ZFIsCEG9T9Y=,-0.134191,0.033446,-0.016809,-0.065544,-0.005454,-0.023188,-0.027744,0.038886,-0.095299,-0.039443,0.078822,-0.156742,0.098338,0.111954,0.055073,0.001971,-0.004374,-0.064540,0.015502,0.196452,0.111543,-0.103727,-0.011391,-0.116658,0.017688,-0.095074,-0.082762,0.167728,0.036701,0.049694,-0.159930,0.058359,0.054372,0.101311,0.020246,0.048172,0.004470,0.082905,-0.123100,0.043494,-0.062408,0.010581,-0.022152,0.090399,0.033835,-0.009878,-0.137972,-0.090482,-0.084104,-0.070353,0.163239,-0.069225,0.128696,-0.062097,-0.043590,-0.081127,0.136361,0.063025,0.004526,-0.005074,-0.066822,-0.081855,-0.040986,0.032930
TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=,-0.108911,0.046885,-0.009206,-0.035942,0.003834,-0.020538,-0.053021,0.053391,-0.065146,-0.065877,0.071320,-0.144882,0.117320,0.127503,0.057279,-0.004522,-0.050809,-0.094180,0.024795,0.190301,0.074600,-0.034318,-0.041415,-0.114379,0.011812,-0.112989,-0.068288,0.158493,0.065175,-0.011483,-0.154075,0.052120,0.058332,0.114753,0.024892,0.047404,-0.012537,0.068243,-0.162484,0.039547,-0.082769,0.030091,-0.003740,0.089420,0.031251,-0.014134,-0.193242,-0.104021,-0.081031,-0.082680,0.154753,-0.067269,0.156542,-0.086332,-0.035301,-0.073855,0.106389,0.063640,0.019810,0.025690,-0.047575,-0.051056,-0.053354,0.015785


In [20]:
# res.to_csv('/content/drive/My Drive/recsys-hw2/user_embeddings.csv')

Сохранил эмбеддинги в .csv, чтобы использовать их в ноутбуке с градиентным бустингом.